In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import os

def load_data():
    
    # Load city coordinates
    cities_df = pd.read_csv('cities_data.csv')
    coords = {}
    for _, row in cities_df.iterrows():
        coords[row['city']] = (row['x'], row['y'])
    
    # Load area assignments
    areas_df = pd.read_csv('areas_data.csv')
    N1 = set(areas_df[areas_df['area'] == 'N1']['city'].tolist())
    N2 = set(areas_df[areas_df['area'] == 'N2']['city'].tolist())
    N3 = set(areas_df[areas_df['area'] == 'N3']['city'].tolist())
    
    return coords, N1, N2, N3

def define_constraints(coords):
    """Define fixed problem constraints"""
    N = set(coords.keys())
    
    # Policy sets (fixed constraints)
    K1 = {"Konin", "Kalisz"} & N           # at least one
    K2 = {"Bytom", "Sosnowiec", "Katowice"} & N  # at most one
    K3 = {"Wodzisław Śląski", "Jastrzębie-Zdrój"} & N  # at most one
    K4 = {"Poznań", "Zielona Góra", "Leszno"} & N      # at least one
    
    # Mandatory edges (fixed infrastructure)
    mandatory_edges = [("Radom", "Kielce"), ("Gliwice", "Opole")]
    
    # Mandatory cities (fixed requirements)
    mandatory_cities = ["Radom", "Kielce", "Gliwice", "Opole"]
    
    return K1, K2, K3, K4, mandatory_edges, mandatory_cities

def euclid(a, b, coords):
    """Calculate Euclidean distance between two cities"""
    ax, ay = coords[a]
    bx, by = coords[b]
    return math.hypot(ax - bx, ay - by)

def solve_telecom_path():
    """Main optimization function"""
    
    # Load variable data from files
    coords, N1, N2, N3 = load_data()
    
    # Define fixed constraints
    K1, K2, K3, K4, mandatory_edges, mandatory_cities = define_constraints(coords)
    
    # Build set of all cities
    N = set(coords.keys())
    
    # Verify data integrity
    assert N1 | N2 | N3 <= N, "Area sets must be subset of N"
    assert (N1 & N2) == set() and (N1 & N3) == set() and (N2 & N3) == set(), "Areas must be disjoint"
    
    # Precompute distances (no self-arcs)
    d = {(i, j): euclid(i, j, coords) for i in N for j in N if i != j}
    
    # Create optimization model
    m = gp.Model("Telecom_Path_SW_Poland")
    m.setParam('OutputFlag', 1)  # Enable output
    
    # Big-M for MTZ ordering
    M = len(N)
    
    # Decision variables
    x = m.addVars(d.keys(), vtype=GRB.BINARY, name="x")  # Arc selection
    y = m.addVars(N, vtype=GRB.BINARY, name="y")        # City inclusion
    u = m.addVars(N, lb=0.0, ub=M, vtype=GRB.CONTINUOUS, name="u")  # Position ordering
    
    # Objective: minimize total cable length
    m.setObjective(gp.quicksum(d[i, j] * x[i, j] for (i, j) in d), GRB.MINIMIZE)
    
    # Constraints
    
    # 1) Start/End constraints
    m.addConstr(gp.quicksum(x["Warsaw", j] for j in N if j != "Warsaw") == 1, "start_out")
    m.addConstr(gp.quicksum(x[i, "Warsaw"] for i in N if i != "Warsaw") == 0, "start_in")
    m.addConstr(gp.quicksum(x[i, "Jelenia Góra"] for i in N if i != "Jelenia Góra") == 1, "end_in")
    m.addConstr(gp.quicksum(x["Jelenia Góra", j] for j in N if j != "Jelenia Góra") == 0, "end_out")
    
    # Flow conservation for intermediate nodes
    for j in N - {"Warsaw", "Jelenia Góra"}:
        m.addConstr(gp.quicksum(x[i, j] for i in N if i != j) == y[j], f"in_{j}")
        m.addConstr(gp.quicksum(x[j, k] for k in N if k != j) == y[j], f"out_{j}")
    
    # City inclusion for endpoints
    m.addConstr(y["Warsaw"] == 1, "y_warsaw")
    m.addConstr(y["Jelenia Góra"] == 1, "y_jg")
    
    # 2) Mandatory cities (fixed business requirements)
    for city in mandatory_cities:
        if city in N:
            m.addConstr(y[city] == 1, f"mandatory_{city}")
    
    # 3) Area coverage counts
    m.addConstr(gp.quicksum(y[i] for i in N1) == 4, "area1_count")
    m.addConstr(gp.quicksum(y[i] for i in N2) == 5, "area2_count")
    m.addConstr(gp.quicksum(y[i] for i in N3) == 5, "area3_count")
    
    # 4) Policy constraints (fixed business rules)
    if K1: m.addConstr(gp.quicksum(y[i] for i in K1) >= 1, "policy_K1")
    if K2: m.addConstr(gp.quicksum(y[i] for i in K2) <= 1, "policy_K2")
    if K3: m.addConstr(gp.quicksum(y[i] for i in K3) <= 1, "policy_K3")
    if K4: m.addConstr(gp.quicksum(y[i] for i in K4) >= 1, "policy_K4")
    
    # 5) No bidirectional connections
    for i in N:
        for j in N:
            if i < j and (i, j) in x and (j, i) in x:
                m.addConstr(x[i, j] + x[j, i] <= 1, f"nobidir_{i}_{j}")
    
    # 6) Edge-activation implies node-activation
    for (i, j) in d:
        m.addConstr(x[i, j] <= y[i], f"x_le_yi_{i}_{j}")
        m.addConstr(x[i, j] <= y[j], f"x_le_yj_{i}_{j}")
    
    # 7) Mandatory existing infrastructure edges
    for (a, b) in mandatory_edges:
        if (a in N) and (b in N) and (a, b) in x:
            m.addConstr(x[a, b] == 1, f"mandatory_edge_{a}_{b}")
    
    # 8) MTZ subtour elimination
    m.addConstr(u["Warsaw"] == 1, "u_start")
    for i in N - {"Warsaw"}:
        m.addConstr(u[i] >= y[i], f"u_lb_{i}")
        m.addConstr(u[i] <= M * y[i], f"u_ub_{i}")
    
    for i in N:
        for j in N:
            if i != j and (i, j) in x:
                m.addConstr(u[i] - u[j] + M * x[i, j] <= M - 1, f"mtz_{i}_{j}")
    
    # Solve the model
    print("Optimizing telecommunications path...")
    m.optimize()
    
    # Extract and display results
    if m.status == GRB.OPTIMAL:
        print("\n" + "="*60)
        print("OPTIMAL SOLUTION FOUND")
        print("="*60)
        
        # Extract solution
        total_length = m.objVal
        selected_nodes = [i for i in N if y[i].X > 0.5]
        
        # Build successor map and reconstruct path
        succ = {i: None for i in N}
        selected_edges = []
        for (i, j), var in x.items():
            if var.X > 0.5:
                succ[i] = j
                selected_edges.append((i, j, d[i, j]))
        
        # Reconstruct path starting from Warsaw
        path = ["Warsaw"]
        while succ[path[-1]] is not None:
            path.append(succ[path[-1]])
            if len(path) > len(N):  # Safety check
                break
        
        # Required outputs
        print(f"\nLength of the main cable: {total_length:.3f} units")
        print(f"\nCities connected to the main cable (in order from Warsaw):")
        for i, city in enumerate(path, 1):
            print(f"  {i:2d}. {city}")
        
        # Additional information
        print(f"\nTotal cities selected: {len(selected_nodes)}")
        print(f"Path segments:")
        for i in range(len(path) - 1):
            from_city, to_city = path[i], path[i + 1]
            segment_length = d[from_city, to_city]
            print(f"  {from_city} -> {to_city}: {segment_length:.3f} units")
        
        # Verify area constraints
        print(f"\nArea distribution:")
        print(f"  Area N1: {len([c for c in selected_nodes if c in N1])} cities")
        print(f"  Area N2: {len([c for c in selected_nodes if c in N2])} cities") 
        print(f"  Area N3: {len([c for c in selected_nodes if c in N3])} cities")
        
        return path, total_length
        
    else:
        print(f"\nOptimization failed. Status: {m.status}")
        if m.status == GRB.INFEASIBLE:
            print("The problem is infeasible - no solution satisfies all constraints.")
        elif m.status == GRB.UNBOUNDED:
            print("The problem is unbounded.")
        else:
            print("Check Gurobi documentation for status code meaning.")
        return None, None

if __name__ == "__main__":
    # Check if data files exist
    required_files = ['cities_data.csv', 'areas_data.csv']
    for file in required_files:
        if not os.path.exists(file):
            print(f"Error: Required data file '{file}' not found!")
            print("Please ensure all data files are in the same directory as this script.")
            exit(1)
    
    # Solve the optimization problem
    path, length = solve_telecom_path()

Set parameter Username
Set parameter LicenseID to value 2711525
Academic license - for non-commercial use only - expires 2026-09-22
Set parameter OutputFlag to value 1
Optimizing telecommunications path...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.6.0 23H626)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2393 rows, 702 columns and 6693 nonzeros
Model fingerprint: 0xfde8ce83
Variable types: 26 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 6e+01]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 1637 rows and 161 columns
Presolve time: 0.03s
Presolved: 756 rows, 541 columns, 3135 nonzeros
Variable types: 22 continuous, 519 integer (519 binary)
Found heuristic solution: objective 447.9609584
Found heuristic solution: objective 341.1510616

Root relaxation: objective 1.327592e+

In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import os

def load_task2_data(main_line_cities):
    """Load data for Task 2 optimization"""
    
    # Load city data with capacities
    cities_df = pd.read_csv('cities_data.csv')
    
    # Extract coordinates and capacities
    coords = {}
    capacities = {}
    
    for _, row in cities_df.iterrows():
        city = row['city']
        coords[city] = (row['x'], row['y'])
        capacities[city] = row['capacity_tb']
    
    # Identify remaining cities (not on main line)
    all_cities = set(coords.keys())
    remaining_cities = all_cities - set(main_line_cities)
    
    # Filter to only cities with capacity requirements > 0
    remaining_cities = {c for c in remaining_cities if capacities[c] > 0}
    
    return coords, capacities, remaining_cities

def euclid(a, b, coords):
    """Calculate Euclidean distance between two cities"""
    ax, ay = coords[a]
    bx, by = coords[b]
    return math.hypot(ax - bx, ay - by)

def solve_copper_network(main_line_cities):
    """Solve Task 2: Connect remaining cities to switching stations"""
    
    # Load data
    coords, capacities, remaining_cities = load_task2_data(main_line_cities)
    
    # Define switching stations and their capacities
    stations = {
        'Warsaw': 800,   # 2 units × 400 TB/s
        'Kielce': 1200,  # 3 units × 400 TB/s
        'Opole': 400     # 1 unit × 400 TB/s
    }
    
    # Parameters
    alpha = 1000  # EUR per coordinate unit per TB/s
    beta = 200    # Maximum capacity per cable (TB/s)
    
    R = remaining_cities  # Set of remaining cities
    S = set(stations.keys())  # Set of switching stations
    
    print("="*70)
    print("TASK 2: COPPER CABLE NETWORK OPTIMIZATION")
    print("="*70)
    print(f"\nRemaining cities to connect: {len(R)}")
    print(f"Cities: {sorted(R)}")
    print(f"\nSwitching stations:")
    for station, cap in stations.items():
        print(f"  {station}: {cap} TB/s capacity")
    
    # Precompute distances from remaining cities to stations
    distances = {}
    for i in R:
        for j in S:
            distances[i, j] = euclid(i, j, coords)
    
    # Create optimization model
    m = gp.Model("Copper_Cable_Network")
    m.setParam('OutputFlag', 1)
    
    # Decision variables
    x = m.addVars(R, S, vtype=GRB.BINARY, name="x")  # Connection indicator
    Q = m.addVars(R, S, lb=0, vtype=GRB.CONTINUOUS, name="Q")  # Flow (TB/s)
    
    # Objective: minimize total cost
    m.setObjective(
        gp.quicksum(alpha * distances[i, j] * Q[i, j] for i in R for j in S),
        GRB.MINIMIZE
    )
    
    # Constraints
    
    # 1. Demand satisfaction: each city must send all its required capacity
    for i in R:
        m.addConstr(
            gp.quicksum(Q[i, j] for j in S) == capacities[i],
            f"demand_{i}"
        )
    
    # 2. Cable capacity limits: max 200 TB/s per cable
    for i in R:
        for j in S:
            m.addConstr(Q[i, j] <= beta * x[i, j], f"cable_cap_{i}_{j}")
    
    # 3. Switching station capacity: total inflow ≤ station capacity
    for j in S:
        m.addConstr(
            gp.quicksum(Q[i, j] for i in R) <= stations[j],
            f"station_cap_{j}"
        )
    
    # 4. Shannon's Law: capacity decreases with distance
    for i in R:
        for j in S:
            d_ij = distances[i, j]
            max_capacity = 2 * math.pi * min(35, 70 - d_ij)
            m.addConstr(
                Q[i, j] <= max_capacity * x[i, j],
                f"shannon_{i}_{j}"
            )
    
    # 5. Łódź special constraint: max 40% on any single cable
    if 'Łódź' in R:
        lodz_capacity = capacities['Łódź']
        for j in S:
            m.addConstr(
                Q['Łódź', j] <= 0.4 * lodz_capacity,
                f"lodz_limit_{j}"
            )
    
    # 6. Connection requirement: each city connects to at least one station
    for i in R:
        m.addConstr(
            gp.quicksum(x[i, j] for j in S) >= 1,
            f"connect_{i}"
        )
    
    # Solve
    print("\nOptimizing copper cable network...")
    m.optimize()
    
    # Extract and display results
    if m.status == GRB.OPTIMAL:
        print("\n" + "="*70)
        print("OPTIMAL SOLUTION FOUND")
        print("="*70)
        
        total_cost = m.objVal
        print(f"\nTotal cost to connect remaining cities: {total_cost:,.2f} EUR")
        
        # Detailed connection information
        print("\n" + "-"*70)
        print("CONNECTION DETAILS")
        print("-"*70)
        
        connections = []
        for i in R:
            for j in S:
                if Q[i, j].X > 0.01:  # Threshold for numerical tolerance
                    connections.append({
                        'city': i,
                        'station': j,
                        'flow': Q[i, j].X,
                        'distance': distances[i, j],
                        'cost': alpha * distances[i, j] * Q[i, j].X
                    })
        
        # Sort by city name
        connections.sort(key=lambda x: x['city'])
        
        for conn in connections:
            print(f"\n{conn['city']} -> {conn['station']}:")
            print(f"  Flow: {conn['flow']:.2f} TB/s")
            print(f"  Distance: {conn['distance']:.3f} units")
            print(f"  Cost: {conn['cost']:,.2f} EUR")
        
        # Answer specific questions
        print("\n" + "="*70)
        print("ANSWERS TO SPECIFIC QUESTIONS")
        print("="*70)
        
        # Question 1: Total costs
        print(f"\n1. Total costs to connect remaining cities: {total_cost:,.2f} EUR")
        
        # Question 2: Capacity for Poznań and Wałbrzych
        print("\n2. Capacity installed between cities and switching stations:")
        
        for city in ['Poznań', 'Wałbrzych']:
            if city in R:
                print(f"\n   {city} (requires {capacities[city]} TB/s):")
                for j in S:
                    if Q[city, j].X > 0.01:
                        print(f"     -> {j}: {Q[city, j].X:.2f} TB/s")
        
        # Question 3: Kielce utilization
        print("\n3. Switching station utilization:")
        
        station_usage = {}
        for j in S:
            total_flow = sum(Q[i, j].X for i in R)
            utilization = (total_flow / stations[j]) * 100
            station_usage[j] = {
                'flow': total_flow,
                'capacity': stations[j],
                'utilization': utilization
            }
            print(f"\n   {j}:")
            print(f"     Total inflow: {total_flow:.2f} TB/s")
            print(f"     Capacity: {stations[j]} TB/s")
            print(f"     Utilization: {utilization:.2f}%")
        
        # Summary statistics
        print("\n" + "="*70)
        print("SUMMARY STATISTICS")
        print("="*70)
        
        print(f"\nTotal remaining cities connected: {len(R)}")
        print(f"Total cables laid: {sum(1 for i in R for j in S if x[i, j].X > 0.5)}")
        print(f"Average cost per city: {total_cost / len(R):,.2f} EUR")
        
        # Check for cities requiring multiple connections
        multi_connect = [i for i in R if sum(x[i, j].X for j in S) > 1.5]
        if multi_connect:
            print(f"\nCities requiring multiple connections: {', '.join(multi_connect)}")
        
        return connections, station_usage, total_cost
        
    else:
        print(f"\nOptimization failed. Status: {m.status}")
        if m.status == GRB.INFEASIBLE:
            print("The problem is infeasible.")
            m.computeIIS()
            print("\nConflicting constraints:")
            for c in m.getConstrs():
                if c.IISConstr:
                    print(f"  {c.constrName}")
        return None, None, None

if __name__ == "__main__":
    # Check if required files exist
    if not os.path.exists('cities_data.csv'):
        print("Error: 'cities_data.csv' not found!")
        print("Please ensure the data file is in the same directory.")
        exit(1)
    
    # Main line cities from Task 1 solution
    main_line_cities = [
        'Warsaw', 'Radom', 'Kielce', 'Kraków', 'Bielsko', 
        'Jastrzębie-Zdrój', 'Gliwice', 'Opole', 'Sieradz', 
        'Konin', 'Kalisz', 'Leszno', 'Legnica', 'Jelenia Góra'
    ]
    
    print("Main line cities (from Task 1):")
    print(", ".join(main_line_cities))
    print()
    
    # Solve Task 2
    connections, station_usage, total_cost = solve_copper_network(main_line_cities)

Main line cities (from Task 1):
Warsaw, Radom, Kielce, Kraków, Bielsko, Jastrzębie-Zdrój, Gliwice, Opole, Sieradz, Konin, Kalisz, Leszno, Legnica, Jelenia Góra

TASK 2: COPPER CABLE NETWORK OPTIMIZATION

Remaining cities to connect: 12
Cities: ['Bytom', 'Czestochowa', 'Katowice', 'Piotrków', 'Poznań', 'Skierniewice', 'Sosnowiec', 'Wałbrzych', 'Wodzisław Śląski', 'Wrocław', 'Zielona Góra', 'Łódź']

Switching stations:
  Warsaw: 800 TB/s capacity
  Kielce: 1200 TB/s capacity
  Opole: 400 TB/s capacity
Set parameter OutputFlag to value 1

Optimizing copper cable network...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 23.6.0 23H626)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 102 rows, 72 columns and 255 nonzeros
Model fingerprint: 0xbf250e05
Variable types: 36 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [9e+03, 6e+04]
  Bo